In [1]:
pip install torch sentencepiece transformers datasets gradio 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 109.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

# Load the Arabic-to-English dataset
dataset = load_dataset('Helsinki-NLP/tatoeba_mt', 'ara-eng')

README.md:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

tatoeba_mt.py:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

The repository for Helsinki-NLP/tatoeba_mt contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Helsinki-NLP/tatoeba_mt.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


tatoeba-test.ara-eng.tsv:   0%|          | 0.00/938k [00:00<?, ?B/s]

tatoeba-dev.ara-eng.tsv:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10304 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19528 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

# Load pre-trained tokenizers for Arabic and English
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

def preprocess_function(examples):
     # Extract source (Arabic) and target (English) texts
    arabic_texts = examples['sourceString']
    english_texts = examples['targetString']
    
    # Tokenize the Arabic texts (inputs) and English texts (labels)
    model_inputs = tokenizer(arabic_texts, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(english_texts, max_length=128, truncation=True, padding='max_length')
    
    # Add labels to the model inputs
    model_inputs['labels'] = labels['input_ids']
    return model_inputs
# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/10304 [00:00<?, ? examples/s]

Map:   0%|          | 0/19528 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSeq2SeqLM

# Load a pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [5]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [6]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    report_to="none",  # Disables WandB logging
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["validation"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-7-9e791347dfd9>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [8]:
# Rest of your code
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.175700,0.135698
2,0.130400,0.118536
3,0.112200,0.114894


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3663, training_loss=0.18308026758725612, metrics={'train_runtime': 830.8478, 'train_samples_per_second': 70.511, 'train_steps_per_second': 4.409, 'total_flos': 1985901810941952.0, 'train_loss': 0.18308026758725612, 'epoch': 3.0})

**Evaluate the Model**

In [9]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate(tokenized_datasets["test"])

# Print evaluation results
print("Evaluation Results:", eval_results)

Evaluation Results: {'eval_loss': 0.11489422619342804, 'eval_runtime': 40.2302, 'eval_samples_per_second': 256.126, 'eval_steps_per_second': 16.008, 'epoch': 3.0}


**Calculate BLEU Score**

In [10]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00


In [12]:
import sacrebleu

# Move model to GPU
model = model.to("cuda")

# Function to generate translations
def generate_translations(model, tokenizer, dataset):
    translations = []
    references = []
    
    for example in dataset:
        # Tokenize the input Arabic text and move to GPU
        inputs = tokenizer(example["sourceString"], return_tensors="pt", truncation=True, padding=True)
        inputs = {key: value.to("cuda") for key, value in inputs.items()}  # Move inputs to GPU
        
        # Generate translation
        translated_tokens = model.generate(**inputs)
        translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        
        # Append the translation and reference
        translations.append(translated_text)
        references.append([example["targetString"]])  # Wrap in a list for sacrebleu
    
    return translations, references

# Generate translations for the test dataset
translations, references = generate_translations(model, tokenizer, tokenized_datasets["test"])

# Calculate BLEU score
bleu_score = sacrebleu.corpus_bleu(translations, references)
print("BLEU Score:", bleu_score.score)

BLEU Score: 32.46679154750989


**Calculate Perplexity**

In [14]:
import torch

perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))
print("Perplexity:", perplexity.item())

Perplexity: 1.121754765510559


**Calculate Accuracy**

***Word-Level Accuracy***

In [15]:
def calculate_word_level_accuracy(translations, references):
    correct = 0
    total = 0
    
    for pred, ref in zip(translations, references):
        pred_words = pred.split()
        ref_words = ref[0].split()  # References are wrapped in a list
        
        # Count correct words
        for pred_word, ref_word in zip(pred_words, ref_words):
            if pred_word == ref_word:
                correct += 1
        total += len(ref_words)
    
    accuracy = (correct / total) * 100
    return accuracy

# Calculate word-level accuracy
word_accuracy = calculate_word_level_accuracy(translations, references)
print("Word-Level Accuracy:", word_accuracy)

Word-Level Accuracy: 47.17999169907614


***Sequence-Level Accuracy***

In [16]:
def calculate_sequence_level_accuracy(translations, references):
    correct = 0
    
    for pred, ref in zip(translations, references):
        if pred == ref[0]:  # Exact match
            correct += 1
    
    accuracy = (correct / len(references)) * 100
    return accuracy

# Calculate sequence-level accuracy
sequence_accuracy = calculate_sequence_level_accuracy(translations, references)
print("Sequence-Level Accuracy:", sequence_accuracy)

Sequence-Level Accuracy: 21.321816770186334


**Visualize Results**

In [17]:
# Print some example translations
for i in range(5):  # Print first 5 examples
    print(f"Source: {tokenized_datasets['test'][i]['sourceString']}")
    print(f"Reference: {tokenized_datasets['test'][i]['targetString']}")
    print(f"Translation: {translations[i]}")
    print("-" * 50)

Source: فكرنا انه طبيعي لازم يتعاقب.
Reference: We thought it natural that he should be punished.
Translation: We think it's nature. He has to be punished.
--------------------------------------------------
Source: واحد اسمه هنري كال يريد يشوفك.
Reference: Someone named Henry wanted to see you.
Translation: One named Henry Kall wants to see you.
--------------------------------------------------
Source: أ أنت جائع ؟
Reference: Are you hungry?
Translation: Are you hungry?
--------------------------------------------------
Source: استجمع توم ما يكفي من الشجاعة لطلب علاوة .
Reference: Tom summoned up enough courage to ask for a raise.
Translation: Tom greated up enough for a bug.
--------------------------------------------------
Source: استدعيت اللجنة فوراً .
Reference: The committee was summoned at once.
Translation: The committee was immediately approached.
--------------------------------------------------


In [21]:
# Save the model and tokenizer
model.save_pretrained("./arabic_to_english_model")
tokenizer.save_pretrained("./arabic_to_english_tokenizer")

('./arabic_to_english_tokenizer/tokenizer_config.json',
 './arabic_to_english_tokenizer/special_tokens_map.json',
 './arabic_to_english_tokenizer/vocab.json',
 './arabic_to_english_tokenizer/source.spm',
 './arabic_to_english_tokenizer/target.spm',
 './arabic_to_english_tokenizer/added_tokens.json')